# Monetary and Fiscal Policy Coordination Notebook


In [263]:
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.tsa.seasonal as tsa
import statsmodels.api as sm
import datetime as dt
import numpy as np

from lets_plot import *
LetsPlot.setup_html()
from IPython.display import Image
import cairosvg

### Functions I made to make datetime conversion easier.

In [264]:
def strpquarter(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

In [265]:
def strptimer(i):
    i = str(i)
    i = i[0:4] + '-' + i[4:6]
    return pd.to_datetime(i)

## Taylor Rule Deviation:

$i_t = 2 + \pi_t + a(\pi_t-\pi^*)+b(y_t - y_t^*)$

+ From the formula above we need:

    + Interest Rate
    + Inflation Rate
    + Targetted Inflation Rate
    + Potential Output

### Interest Rates:

Sources: https://iftp.chinamoney.com.cn/english/bmklpr/, https://www.bis.org/statistics/cbpol/cbpol.xlsx

I need to quarterlyize the interest rates

In [266]:
# Static Variables

pstar = 3.0
ustar = 4.86

# Getting my dataset for interest rates

df = pd.DataFrame()
for i in range(0,8): # For between 2013-2020.
    t = pd.read_excel(f'../data/interest-rates/Loan_Prime_Rate_Historical_Data ({i}).xlsx',engine='openpyxl')
    t = t.loc[~((t['Date']=='Data source:')|(t['Date'].isnull())),:]
    df = pd.concat([df,t], ignore_index=True)

# Getting the dataset for benchmark rates 2010-2013.
t = pd.read_excel('../data/interest-rates/cbpol.xlsx', engine = 'openpyxl', sheet_name = 'Monthly Series')
t.columns = t.loc[2]
t = t.drop([0,1,2])
t = t.loc[(~(t['M:CN'].isnull())&(t['Period']<dt.datetime(2013,10,25))&(t['Period']>dt.datetime(2009,12,31))),['Period','M:CN']]
t['Date'] = t['Period']
t['1Y'] = t['M:CN']
df = pd.concat([df,t], ignore_index=True)

# Cleaning the data up.
df['Date'] = pd.to_datetime(df['Date'])
df['1Y'] = df['1Y'].astype(float)
df = df.rename(columns={'1Y':'OneYearLPRRate'})
df = df.sort_values(by='Date',axis=0)
df = df.reset_index()
df = df.drop(columns =['Period', 'M:CN','5Y', 'index'])

# Changing to quarterly data
df['Quarter'] = pd.PeriodIndex(df['Date'],freq = 'Q')
df = df.groupby('Quarter', as_index=False).mean()
df['QuarterDate'] = df['Quarter'].apply(strpquarter)
df = df.set_index(df['QuarterDate'])
df = df.drop(columns = ['Date', 'QuarterDate'])

# Inflation Stat
df1 = pd.read_csv('../data/inflation-rates/CPALTT01CNQ659N.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['Inflation'] = df1['CPALTT01CNQ659N']

# GDP Stat
df1 = pd.read_csv('../data/real-GDP/CHNGDPNQDSMEI.csv')
df1['observation_date'] = pd.to_datetime(df1['observation_date'])
df1 = df1.set_index(df1['observation_date'])
df1 = df1.drop(columns=['observation_date'])
df1 = df1['2010-01-01':'2020-12-31']
df['NominalGDP'] = df1['CHNGDPNQDSMEI']
df['NominalGDP']=tsa.STL(df['NominalGDP']).fit().trend

# Unemployment Stat
df1 = pd.read_excel('../data/unemployment-rates/imf-dm-export-modified-20250226.xls')
df1 = df1.transpose()
df1 = df1.reset_index()
df1.columns = df1.loc[0]
df1 = df1.rename(columns={'Unemployment Rate':'UnemploymentRate'})
df1 = df1.drop([0])
df1['Year'] = pd.to_datetime(df1['Year'], format='%Y')
df1 = df1.set_index(df1['Year'])
df1 = df1.drop(columns=['Year'])
df['UnemploymentRate'] = df1['UnemploymentRate']
df['UnemploymentRate'] = df['UnemploymentRate'].fillna(method='ffill')

# Ystar
df['Ystar'] = df['NominalGDP']/(1-0.4*(df['UnemploymentRate']-ustar))
#gdp_trend, gdp_cycle = sm.tsa.filters.hpfilter(df["NominalGDP"], lamb=1600)
#output_gap = (df["NominalGDP"] - gdp_trend) / gdp_trend * 100
#df['Ystar'] = df['NominalGDP'] + output_gap
#df["OutputGap"] = output_gap

# istar
df['OutputGap'] = (df['NominalGDP']-df['Ystar'])/df['Ystar']
df['Istar'] = 2 + 1.5*df['Inflation'] + 0.5*df['OutputGap']+1

# Taylor Rule Deviations
df['TaylorDeviation'] = df['OneYearLPRRate']-df['Istar']


# Debt and Primary Expenditure for Fiscal Policy Response Indicator
df1 = pd.read_excel('../data/government-spending/imf-dm-export-20250226 (1).xls')
df1 = df1.transpose()
df1 = df1.reset_index()
df1.columns = df1.loc[0]
df1 = df1.rename(columns = {'Government primary expenditure, percent of GDP (% of GDP)':'PrimExp',
                            'General government gross debt (Percent of GDP)':'Debt'})
df1 = df1.drop([0])
df1['Year'] = pd.to_datetime(df1['Year'], format = '%Y')
df1 = df1.set_index('Year')

df['Debt'] = -df1['Debt']
df['DebtMinusHalf'] = df['Debt'].shift(2)
df['PrimExp'] = df1['PrimExp']
df['Debt'] = df['Debt'].fillna(method='ffill')
df['PrimExp'] = df['PrimExp'].fillna(method='ffill')
df['DebtMinusHalf'] = df['DebtMinusHalf'].fillna(method='ffill')
df.loc['2010-01-01':'2010-04-01','DebtMinusHalf'] = df.loc['2010-07-01','DebtMinusHalf']

df.to_csv('../data/chinese_coordinator.csv')
df

c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\ellem\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl'

,Quarter,OneYearLPRRate,Inflation,NominalGDP,UnemploymentRate,Ystar,OutputGap,Istar,TaylorDeviation,Debt,DebtMinusHalf,PrimExp
QuarterDate,,,,,,,,,,,,
2010-01-01,2010Q1,5.310000,2.071556,9.556961e+12,4.10,7.328958e+12,0.304,6.259334,-0.949334,-33.9,-33.9,-0.420500
2010-04-01,2010Q2,5.310000,2.718078,1.003549e+13,4.10,7.695926e+12,0.304,7.229117,-1.919117,-33.9,-33.9,-0.420500
2010-07-01,2010Q3,5.310000,3.264564,1.051467e+13,4.10,8.063399e+12,0.304,8.048846,-2.738846,-33.9,-33.9,-0.420500
2010-10-01,2010Q4,5.643333,4.642333,1.099267e+13,4.10,8.429963e+12,0.304,10.115499,-4.472166,-33.9,-33.9,-0.420500
2011-01-01,2011Q1,5.976667,5.223647,1.147361e+13,4.10,8.798778e+12,0.304,10.987471,-5.010804,-33.8,-33.9,-0.005371
2011-04-01,2011Q2,6.310000,5.954095,1.192189e+13,4.10,9.142557e+12,0.304,12.083143,-5.773143,-33.8,-33.9,-0.005371
2011-07-01,2011Q3,6.560000,6.438447,1.231944e+13,4.10,9.447427e+12,0.304,12.809670,-6.249670,-33.8,-33.8,-0.005371
2011-10-01,2011Q4,6.560000,4.620924,1.267303e+13,4.10,9.718579e+12,0.304,10.083386,-3.523386,-33.8,-33.8,-0.005371
2012-01-01,2012Q1,6.560000,3.727297,1.298816e+13,4.10,9.960245e+12,0.304,8.742946,-2.182946,-34.4,-33.8,0.134427


In [267]:
plot3 = (
    ggplot(data = df.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'NominalGDP'), manual_key = 'GDP', color = 'red')
    + geom_line(aes(y = 'Ystar'), manual_key = 'Potential GDP', color = 'black')
    + theme(legend_position = 'bottom',
            plot_caption=element_text(color='grey',size=9,hjust=0),
            plot_caption_position='panel'
            )
    + labs(y = 'Output',
           x='Date',
           title = '',
           caption = f"""Nominal GDP has been seasonally adjusted from the original data set. Since potential output is a function of unemployment 
           due to being calculated through Okun's law, output potential indirectly illustrates the effect of unemployment on the rest of 
           the variables.""")
    + scale_y_continuous(format = 's')
)
ggsave(plot3, 'plot3.png', path= '../images')
plot3

In [268]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/20
plot['Debt'] = plot['Debt']
plot2 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'OutputGap'), manual_key = 'Output Gap', color = 'red')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    #+ geom_line(aes(y = 'Debt'), manual_key = 'Public Debt', color = 'blue')
    + theme_classic()
    + theme(legend_position='top',
            plot_title=element_text(face='bold'),
            axis_text_y = element_blank(),
            plot_subtitle = element_text(color = 'grey'),
            plot_caption = element_text(color='grey')
            )
    + labs(y = 'Normalized Axis', 
           x = '',
           title = 'Primary Expenditure (Measured) and Output Gap (Calculated) Strongly Covary with one Another',
           subtitle = 'Their positive covariance also suggests that Chinese fiscal spending is counter-cyclical.',
           caption = 'Axes are exaggerated for clearer illustration of covariance.'
           )
    + ggsize(1000,500)
    
)
ggsave(plot2, 'plot2.png', path= '../images')
plot2

In [269]:
formula = 'PrimExp ~ DebtMinusHalf + OutputGap'
model1 = smf.ols(formula = formula, data = df).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:                PrimExp   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     290.5
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           6.15e-25
Time:                        13:42:35   Log-Likelihood:                -38.631
No. Observations:                  44   AIC:                             83.26
Df Residuals:                      41   BIC:                             88.61
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -2.8721      1.577     -1.821

In [270]:
formula = 'OneYearLPRRate ~ Inflation + OutputGap'
model = smf.ols(formula = formula, data = df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         OneYearLPRRate   R-squared:                       0.844
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     111.1
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           2.81e-17
Time:                        13:42:35   Log-Likelihood:                -15.770
No. Observations:                  44   AIC:                             37.54
Df Residuals:                      41   BIC:                             42.89
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.2222      0.120     35.161      0.0

In [271]:
plot = df.copy()
plot['PrimExp'] = plot['PrimExp']/2 + 4
plot1 = (
    ggplot(data = plot.reset_index(), mapping = aes(x = 'QuarterDate'))
    + geom_line(aes(y = 'OneYearLPRRate'), manual_key = '1 Year LPR', color = 'blue')
    + geom_line(aes(y = 'PrimExp'), manual_key = 'Primary Expenditure')
    + theme_classic()
    + labs(x = '', 
           y = 'Normalized Axis', 
           title = 'The 1 Year LPR and Primary Expenditure Show High Positive Covariance', 
           subtitle = 'This signifies that Chinese Monetary Policy and Chinese Fiscal Policy do work Hand-in-hand.',
           caption = f'Primary Expenditure and 1 Year LPR have a correlation coefficient of {np.corrcoef(df['PrimExp'],df['OneYearLPRRate'])[0,1]}')
    + theme(plot_title = element_text(face = 'bold'),
            plot_subtitle = element_text(color = 'grey'),
            legend_position='top',
            plot_caption = element_text(color='grey'),
            axis_text_y= element_blank()
            )
    + ggsize(1000,500)
)
ggsave(plot1, 'plot1.png', path= '../images')
plot1